In [1]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [2]:
#data prefix for output data
yearmonth = "202404"

In [3]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [4]:
#url
base_url_list = \
[
    ['Child Care',"https://gbph-portal.hedgerowsoftware.com/#/6f62eef5-6e2d-4046-bcd5-755d2287d990/disclosure", ['Facility Name','Site Address','Last Inspection Date','Facility Type'] ],
    ['Food Safety',"https://gbph-portal.hedgerowsoftware.com/#/b67cdb25-e9dc-4a9c-beba-3fc076733c84/disclosure", ['Facility Name','Site Address','Last Inspection Date']],
    ['Person Service Setting',"https://gbph-portal.hedgerowsoftware.com/#/e842aa6d-ee44-4e9e-b6af-1712da884a2e/disclosure", ['Facility Name','Site Address','Last Inspection Date']],
    ['Public Beach',"https://gbph-portal.hedgerowsoftware.com/#/c0cc2867-a863-42a3-80bc-61462391044c/disclosure",['Public Beach','Address','Date of Status Update']],
    ['Recreation Camp',"https://gbph-portal.hedgerowsoftware.com/#/e230a121-49a8-455b-a1b6-12a99f986cb8/disclosure",['Facility Name','Site Address','Last Inspection Date']],
    ['Recreational Water',"https://gbph-portal.hedgerowsoftware.com/#/50bdcd35-4ad2-4f67-9d14-0647ea610f3d/disclosure",['Facility Name','Site Address','Last Inspection Date']],
    ['Small Drinking Water',"https://gbph-portal.hedgerowsoftware.com/#/2b65a9c9-28d5-4a94-ab97-41298b7a54b8/disclosure",['Facility Name','Site Address','Last Inspection Date']],
    ['Tanning',"https://gbph-portal.hedgerowsoftware.com/#/cab86cf0-184a-4874-a8be-5bc1ea9d5b35/disclosure",['Facility Name','Site Address','Last Inspection Date']],                                                                                                             
    ['Enforcements',"https://gbph-portal.hedgerowsoftware.com/#/541999d0-2d8e-4e01-bb0c-9aa1d0addba1/disclosure",['Facility Name','Site Address','Type', 'Legislation','Offense/Issue Date', 'Status','Resolved Date']],                                                                                                                       
    ['Water Advisories',"https://gbph-portal.hedgerowsoftware.com/#/ce3b1b55-85a0-4cfd-8d52-f830d398d1c4/disclosure",['SDWS Name','Address','Advisory Type', 'Date Issued', 'Date Lifted']],                                                                                                                       
]

    

In [5]:
base_url_list

[['Child Care',
  'https://gbph-portal.hedgerowsoftware.com/#/6f62eef5-6e2d-4046-bcd5-755d2287d990/disclosure',
  ['Facility Name', 'Site Address', 'Last Inspection Date', 'Facility Type']],
 ['Food Safety',
  'https://gbph-portal.hedgerowsoftware.com/#/b67cdb25-e9dc-4a9c-beba-3fc076733c84/disclosure',
  ['Facility Name', 'Site Address', 'Last Inspection Date']],
 ['Person Service Setting',
  'https://gbph-portal.hedgerowsoftware.com/#/e842aa6d-ee44-4e9e-b6af-1712da884a2e/disclosure',
  ['Facility Name', 'Site Address', 'Last Inspection Date']],
 ['Public Beach',
  'https://gbph-portal.hedgerowsoftware.com/#/c0cc2867-a863-42a3-80bc-61462391044c/disclosure',
  ['Public Beach', 'Address', 'Date of Status Update']],
 ['Recreation Camp',
  'https://gbph-portal.hedgerowsoftware.com/#/e230a121-49a8-455b-a1b6-12a99f986cb8/disclosure',
  ['Facility Name', 'Site Address', 'Last Inspection Date']],
 ['Recreational Water',
  'https://gbph-portal.hedgerowsoftware.com/#/50bdcd35-4ad2-4f67-9d14-0647

In [6]:
#place holder for final dataframe list
df_list = []

for u in base_url_list:
    category = u[0]
    base_url = u[1]
    columns  = u[2]

    #load main landing page
    driver.get(base_url)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")

        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass
    
    # We will loop through until there's no next page button on the webpage
    #find next page button
    last_page = False

    while last_page==False:
        try:
            next_btn = driver.find_element(By.CLASS_NAME, value="next-page-link")
        except:
            last_page = True


        #place holder for extracted data
        data_url_list = []

        #locate each table rows
        data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

        for row in data_rows:
            data_url_list.append( row.get_attribute("onclick") )

        #convert html table into dataframe
        df = pd.read_html(driver.page_source)
        df = df[0]
        

        #rename columns for clean up
        df.columns = columns
        
        #append url column and category column
        df['url'] = data_url_list
        df['category'] = category

        #put aside resulting table
        df_list.append(df)

        if last_page==True:
            break
        #move to the next page
        next_btn.click()

/var/folders/4k/pwnwdlj13ng6pbf991lxty440000gn/T/ipykernel_7688/4001557055.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(driver.page_source)


ValueError: No tables found

In [ ]:
final_df = pd.concat(df_list)

In [ ]:
final_df

In [ ]:
#serialize the dataset
final_df.to_csv( "../data/{}_1st_table.csv".format(yearmonth) )

In [ ]:
#close webdriver
try:
    driver.close()
except:
    pass